# Dataset
This dataset contains Amazon mobile phone reviews with around 5000 entries for each unique rating summing to 26470 entries.


In [6]:
from google.colab import files
uploaded = files.upload()

Saving amanew.csv to amanew.csv


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
import io
df = pd.read_csv(io.BytesIO(uploaded['amanew.csv']))

In [9]:
df2=df.dropna()

# Pre Processing
Added a new column (Sentiment) with the dataframe.Then splitted the dataframe for training & testing.Then cleaned the review text for tokenization and converting to sequence.After that, encoded the sentiment values.

In [10]:
sentiment=[]
for i in df2['Rating']:
  if i<3:
    sentiment.append(0)  #sentiment=0 for rating<3
  elif i>3:
    sentiment.append(1)  #sentiment=0.5 for rating=3
  else:
    sentiment.append(0.5)  #sentiment=1 for rating>3


In [11]:
df2['Sentiment']=sentiment
df2.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(26470, 3)

In [49]:
from sklearn.model_selection import train_test_split
# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(df2['Reviews'], df2['Sentiment'], \
                                                    test_size=0.1)

In [50]:
from bs4 import BeautifulSoup  
import re
import nltk
from nltk.corpus import stopwords 
from nltk.stem import SnowballStemmer

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense,Activation,Dropout
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM,GRU
from keras.preprocessing.text import Tokenizer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [51]:
def cleanText(raw_text):
    text = BeautifulSoup(raw_text, 'lxml').get_text()  #remove html
    letters_only = re.sub("[^a-zA-Z]", " ", text)  # remove non-character
    words = letters_only.lower().split() # convert to lower case 
    #stops = set(stopwords.words("english"))
    #words = [w for w in words if not w in stops]

    #removing stop words resulted in reduced accuracy
    
    #stemmer = SnowballStemmer('english') 
    #words = [stemmer.stem(w) for w in words]    
    
    #stemming results in reduced accuracy, so not used
                                                  
    
    return( " ".join(words))

In [52]:
X_train_cleaned = []
X_test_cleaned = []


for d in X_train:
    X_train_cleaned.append(cleanText(d))   
for d in X_test:
    X_test_cleaned.append(cleanText(d))
   

In [53]:
# Vectorize X_train and X_test to 2D tensor
tokenizer = Tokenizer(num_words=20000) #only consider top 20000 words in the corpse
tokenizer.fit_on_texts(X_train)
# tokenizer.word_index #access word-to-index dictionary of trained tokenizer

sequences_train = tokenizer.texts_to_sequences(X_train_cleaned)
sequences_test = tokenizer.texts_to_sequences(X_test_cleaned)

X_train_seq = sequence.pad_sequences(sequences_train, maxlen=500)
X_test_seq = sequence.pad_sequences(sequences_test, maxlen=500)



In [54]:
def encoding(dataset):
  enc_list=[]
  for i in dataset:
    if i==0:
      enc_list.append([1,0,0])
    elif i==0.5:
      enc_list.append([0,1,0])
    else:
      enc_list.append([0,0,1]) 
  return np.array(enc_list)


In [55]:
y_train_seq=encoding(y_train)
y_test_seq=encoding(y_test)


# Model Architecture
used both LSTM & GRU, GRU performed better

In [58]:
model1 = Sequential()
model1.add(Embedding(20000, 128))
model1.add(GRU(128)) #used both GRU & LSTM, GRU performed better
model1.add(Dense(3))
model1.add(Activation('softmax'))
model1.summary()

#used both binary & categorical croos entropy, categorical performed better 
model1.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

history=model1.fit(X_train_seq, y_train_seq,validation_split=0.22, batch_size=64,epochs=6, verbose=1)

# Model evluation
score = model1.evaluate(X_test_seq, y_test_seq, batch_size=32)
print('Test loss : {:.4f}'.format(score[0]))
print('Test accuracy : {:.4f}'.format(score[1]))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
gru_4 (GRU)                  (None, 128)               99072     
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 387       
_________________________________________________________________
activation_4 (Activation)    (None, 3)                 0         
Total params: 2,659,459
Trainable params: 2,659,459
Non-trainable params: 0
_________________________________________________________________
Epoch 1/6
291/291 [==============================] - 257s 877ms/step - loss: 0.8548 - accuracy: 0.6239 - val_loss: 0.5776 - val_accuracy: 0.7795
Epoch 2/6
291/291 [==============================] - 254s 874ms/step - loss: 0.4402 - accuracy: 0

# Predicting new reviews

In [ ]:
def predict(sent):
  cleaned_new=cleanText(sent)
  sequences_train_new = tokenizer.texts_to_sequences([cleaned_new])
  X_train_seq_new = sequence.pad_sequences(sequences_train_new, maxlen=100)
  predict_array=model1.predict(X_train_seq_new)
  index=np.argmax(predict_array)
  if index==0:
    return 'negative'
  elif index==1:
    return 'neutral'
  else:
    return 'positive'     


In [ ]:
sent=predict("ok")
sent

'neutral'